# Scraping text from bible.is

In [1]:
import os, requests, re, time
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
root_url = 'https://live.bible.is'

In [3]:
#New Testament
#fr ref  : 1910 Louis Segond (Tresorsonore recording)
#eng ref : King James version 

#change this to the language you want
#language = ['/bible/ENGKJV', '/bible/FRNTLS']
language =['/bible/FRNTLS']

In [9]:
# New testament books

book = ['MAT', 'MRK', 'LUK', 'JHN', 'ACT', 'ROM', '1CO', '2CO', 'GAL', 'EPH', 'PHP', 'COL',
        '1TH', '2TH', '1TI', '2TI', 'TIT', 'PHM', 'HEB', 'JAS', '1PE', '2PE', '1JN', '2JN',
        '3JN', 'JUD', 'REV']

#Old + New testament books

'''
book = ["GEN", "EXO", "LEV", "NUM", "DEU", "JOS", "JDG", "RUT", "1SA", "2SA", "1KI", "2KI",
        "1CH", "2CH", "EZR", "NEH", "EST", "JOB", "PSA", "PRO", "ECC", "SNG", "ISA", "JER",
        "LAM", "EZK", "DAN", "HOS", "JOL", "AMO", "OBA", "JON", "MIC", "NAM", "HAB", "ZEP",
        "HAG", "ZEC", "MAL",'MAT', 'MRK', 'LUK', 'JHN', 'ACT', 'ROM', '1CO', '2CO', 'GAL',
        'EPH', 'PHP', 'COL','1TH', '2TH', '1TI', '2TI', 'TIT', 'PHM', 'HEB', 'JAS', '1PE',
        '2PE', '1JN', '2JN','3JN', 'JUD', 'REV']
'''

In [8]:
# create a new directory
try:
    os.mkdir(f"{os.curdir}"+"/text_Data/")
except FileExistsError:
    pass
# set the new directory as the current directory
os.chdir(f"{os.curdir}"+"/text_Data/")

cur_w_d = os.getcwd()

In [13]:
# if you run the cell below, go and check the output at this path

cur_w_d

'C:\\Users\\Jamiil\\Documents\\Challenge\\AI4D-African Language Dataset Challenge\\text_Data'

In [ ]:
%%time
lines = []
for i in tqdm(range(len(language)), desc='lang'):
    try:
        create_lang_dir = cur_w_d+"/"+ language[i].split('/')[-1] +"/"
        os.mkdir(create_lang_dir)
    except FileExistsError:
        pass
    for j in tqdm(range(len(book)), desc='book'):
        urls = []
        url = root_url + language[i] + "/" + book[j] + "/"
        urls.append(url)
        try:
            create_book_dir = cur_w_d+"/"+language[i].split('/')[-1] +"/"+ book[j]+"/" 
            os.mkdir(create_book_dir)
        except FileExistsError:
            pass
        
        #book_of_nt = []
        for url_item in urls:
            for k in tqdm(range(1,30), desc='chapter'): # Set the max number of chapter per book to 30 
                page = url_item + str(k)
                #print(page)
                response = requests.get(page)
                if response.url == page :
                    #print(response) # display 200 meaning that we successfully access the link
                    soup = BeautifulSoup(response.content, "html.parser")
                    verses = soup.find_all("span", {"class": ""})
                    verses_of_nt = [verse.text for verse in verses]
                    verses_of_nt_prep = verses_of_nt[:-15] # a little preprocessing step
                    #book_of_nt.append(verses_of_nt_prep)
                    
                    create_chapter_file_name = cur_w_d+"/"+language[i].split('/')[-1] +"/"+ book[j]+"/"+book[j]+"_chap_"+str(k)

                    with open(create_chapter_file_name+'.txt', 'a') as file: # specified the name of the file
                        for verse_no, line in enumerate(iterable=verses_of_nt_prep, start=1):
                            file.write("( "+book[j]+" "+str(k)+":" +str(verse_no)+" ), ")
                            if line !="-":
                                file.write(line)
                                lines.append(line) # for further preprocessing steps
                            else:
                                file.write("Interlude")
                                lines.append("Interlude") #for further preprocessing steps
                            file.write('\n')
                            
                else :
                    #print("<Response [404]>") # Because links don't overflow we instead get back to square 1
                    break
    print("Youpi! 1 minute break \n",30*"=")
    time.sleep(60)
print("DONE")